# 0. Install and Import Dependencies

In [ ]:
! pip install opencv-python matplotlib tensorflow imageio gdown 

In [2]:
import os
import cv2
import tensorflow as tf
import numpy as np
from typing import List
from matplotlib import pyplot as plt
import imageio

In [5]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  pass

# 1. Data Loading Functions